In [1]:
import pandas as pd
import numpy as np
import re
from unidecode import unidecode
import os
import glob

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_columns', None)

In [3]:



def primeiros_tratamentos(df):
    ###########primeiros tratamentos
    indice_selecoes = df[df['Capital'] == "{ñ class}"].index[0]
    # Apagar as linhas a partir desse índice até o final
    df = df.iloc[:indice_selecoes]
    #tratando top row que esta deslocada
    df.iloc[0, 1:] = df.iloc[0, :-1].values
    #deletando registros que nao tem informações
    df = df[df[' Faixa Remun Média (SM)'] != '{ñ class}']
    #valores NaN em nulo 
    df = df.replace("NaN", np.nan)
    #preenchendo linhas nan para localidade
    df['Capital'] = df['Capital'].ffill()
    #deletando oq vem antes do nome da capital
    df['Capital'] = df['Capital'].str.split(':').str[1]
    #deletar linhas de totais
    df = df[df[' Faixa Remun Média (SM)'] != 'Total']
    #apagar todas as colunas com total no nome
    df = df.drop(df.filter(like='Total').columns, axis=1)
    #df.columns =  [re.sub(r"^\d+:", "", re.sub(r"\.\d+$", "", coluna)) for coluna in df.columns]
    df.columns = [re.sub(r'^\s*\d+:', '', re.sub(r'\.\d+\s*$', '', coluna)) for coluna in df.columns]
    return df








In [4]:
#separando masculino
def criando_df_masculino(df,ano):
    masculino_cols = ['Capital', ' Faixa Remun Média (SM)'] + [coluna for coluna in df.columns if 'Masculino' in coluna]
    df_m=df[masculino_cols]
    df_m.iloc[0,0]= "capital"
    df_m.iloc[0,1]= "faixa_remuneracao_media_sm"
    new_header = df_m.iloc[0]
    df_m = df_m[1:]
    df_m.columns = new_header 
    #apagando as colunas duplicadas pq misteriosamente elas duplicam muitas vezes 
    df_m = df_m.loc[:, ~df_m.columns.duplicated()]
    df_m["sexo"] = "M"
    df_m["ano"] = ano
    df_m.columns =[re.sub(r"^\d+:", "", coluna) for coluna in df_m.columns]
    df_m = df_m.drop(df_m.filter(like='Total').columns, axis=1)
    return df_m



In [5]:
#separando feminino
def criando_df_feminino(df,ano):
    feminino_cols = ['Capital', ' Faixa Remun Média (SM)'] + [coluna for coluna in df.columns if 'Feminino' in coluna]
    df_f=df[feminino_cols]
    df_f.iloc[0,0]= "capital"
    df_f.iloc[0,1]= "faixa_remuneracao_media_sm"
    new_header = df_f.iloc[0]
    df_f = df_f[1:]
    df_f.columns = new_header 
    #apagando as colunas duplicadas pq misteriosamente elas duplicam muitas vezes 
    df_f = df_f.loc[:, ~df_f.columns.duplicated()]
    df_f["sexo"] = "F"
    df_f["ano"] = ano
    df_f.columns =[re.sub(r"^\d+:", "", coluna) for coluna in df_f.columns]
    df_f = df_f.drop(df_f.filter(like='Total').columns, axis=1)
    return df_f



In [6]:

final_file_name=""
df_final= pd.DataFrame()
for root, dirs, files in os.walk("/Users/bolsolui/Documents/personal/MECAI/git/rais_dados/new_data/"):
    print(f"root:{root}")
    path = root
    extension = 'csv'
    os.chdir(path)
    files_csv = glob.glob('*.{}'.format(extension))
    
    if files_csv:
        
        for i in files_csv:
            resultado = re.search(r'^[^0-9]+', i).group()

            # Remove o último caractere se for um sublinhado
            final_file_name = resultado.rstrip('_')
            
            #print(f"root:{root} e lista: {files_csv} e file : {final_file_name}")
            df = pd.read_csv(i,sep=";",  encoding='latin-1',skiprows=1, header=[0])
            ano = re.search(r'\d{4}', i).group()
            df = primeiros_tratamentos(df)
            df_f = criando_df_feminino(df,ano)
            df_m = criando_df_masculino(df,ano)
            df_final = pd.concat([df_final,df_f, df_m])
            print(f"File {i}, com shape {df.shape} do ano {ano}, df feminino: {df_f.shape}, df masculino: {df_m.shape} e df final: {df_final.shape}")


        
        # Remover acentuação
        df_final['faixa_remuneracao_media_sm'] = df_final['faixa_remuneracao_media_sm'].apply(lambda x: unidecode(str(x)))
        df_final['capital'] = df_final['capital'].apply(lambda x: unidecode(str(x)))
        #tirar espaco 
        df_final['faixa_remuneracao_media_sm'] = df_final['faixa_remuneracao_media_sm'].str.replace(' ', '_')
        df_final['capital'] = df_final['capital'].str.replace('-', '_').str.replace(' ', '_')
        # Transformar em minúsculas
        df_final['faixa_remuneracao_media_sm'] = df_final['faixa_remuneracao_media_sm'].str.lower()
        df_final['capital'] = df_final['capital'].str.lower()
        print(f"ESCREVENDO DF FINAL:   {final_file_name}")
        df_final.to_csv(f'{final_file_name}.csv', index=False)
        print(f"Finalizado:   {final_file_name}")

         

root:/Users/bolsolui/Documents/personal/MECAI/git/rais_dados/new_data/
root:/Users/bolsolui/Documents/personal/MECAI/git/rais_dados/new_data/CNAE_grupo


/Users/bolsolui/Library/Python/3.11/lib/python/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/bolsolui/Library/Python/3.11/lib/python/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


File CNAE_grupo_2014.csv, com shape (325, 439) do ano 2014, df feminino: (324, 221), df masculino: (324, 222) e df final: (648, 222)
File CNAE_grupo_2015.csv, com shape (325, 439) do ano 2015, df feminino: (324, 221), df masculino: (324, 222) e df final: (1296, 222)
File CNAE_grupo_2017.csv, com shape (325, 439) do ano 2017, df feminino: (324, 221), df masculino: (324, 222) e df final: (1944, 222)
File CNAE_grupo_2016.csv, com shape (325, 439) do ano 2016, df feminino: (324, 221), df masculino: (324, 222) e df final: (2592, 222)
File CNAE_grupo_2006.csv, com shape (325, 439) do ano 2006, df feminino: (324, 221), df masculino: (324, 222) e df final: (3240, 222)
File CNAE_grupo_2012.csv, com shape (325, 439) do ano 2012, df feminino: (324, 221), df masculino: (324, 222) e df final: (3888, 222)
File CNAE_grupo_2013.csv, com shape (325, 439) do ano 2013, df feminino: (324, 221), df masculino: (324, 222) e df final: (4536, 222)
File CNAE_grupo_2007.csv, com shape (325, 439) do ano 2007, df 